In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, TensorDataset, DataLoader
import re
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.model_selection import train_test_split

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# loading the datasets
fake_df = pd.read_csv('Fake.csv')
true_df = pd.read_csv('True.csv')

In [ ]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
# adding labels
fake_df['label'] = 1
true_df['label'] = 0

In [ ]:
true_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [ ]:
# now combinning the dataframes
df = pd.concat([fake_df, true_df], ignore_index=True)

In [ ]:
df.sample(5)

,title,text,subject,date,label
2496,GOP Senator DESPERATELY Worried About Trump’s...,"On Thursday afternoon, the world was treated t...",News,"February 16, 2017",1
1602,Heartless Former GOP Lawmaker Had The Most DE...,Former GOP Congressman Joe Walsh (R-Ill) has a...,News,"May 3, 2017",1
16089,CNN Liberal Anchor Freaks Out At Navy SEAL Whe...,A Navy SEAL Carl Higbie was blasted and screec...,Government News,"May 17, 2017",1
34156,Detroit mayor eyes legal action over pension s...,DETROIT (Reuters) - Detroit may sue some of th...,politicsNews,"February 24, 2016",0
20067,ACTOR TIM ALLEN: Hillary Isn’t Capable Of Bein...,Megyn: What about Tim Allen? You re one of th...,left-news,"Aug 27, 2016",1


In [ ]:
# combining title and text for better inputs
df['full_text'] = df['title'] + ' ' + df['text']

In [ ]:
# shuffling the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
print("Dataset preview:")
print(df[['full_text', 'label']].head())

Dataset preview:
                                           full_text  label
0  Ben Stein Calls Out 9th Circuit Court: Committ...      1
1  Trump drops Steve Bannon from National Securit...      0
2  Puerto Rico expects U.S. to lift Jones Act shi...      0
3   OOPS: Trump Just Accidentally Confirmed He Le...      1
4  Donald Trump heads for Scotland to reopen a go...      0


In [ ]:
# Preprosessing and vocabulary

# text cleaning and tokenization
def preprocess_text(text):
  text = text.lower()
  text = re.sub(r'[^a-zA-Z\s]', '', text) # Keep only letters and spaces
  tokens = word_tokenize(text)
  return tokens

In [ ]:
# builiding vocabulary
all_texts = ' '.join(df['full_text'])
all_tokens = preprocess_text(all_texts)
word_counts = Counter(all_tokens)
vocab = sorted(word_counts, key=word_counts.get, reverse=True)

In [ ]:
# word to integer mapping, basically embedding generation
vocab_to_int = {word: i+2 for i, word in enumerate(vocab)}
vocab_to_int['<PAD>'] = 0
vocab_to_int['<UNK>'] = 1

In [ ]:
# encoding
encoded_text = []
for text in df['full_text']:
  tokens = preprocess_text(text)
  encoded = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in tokens]
  encoded_text.append(encoded)

In [ ]:
# pad and truncating the sequence
def pad_features(encoded_text, seq_length):
  features = np.zeros((len(encoded_text), seq_length), dtype=int)
  for i, row in enumerate(encoded_text):
    if len(row) != 0:
      features[i, -len(row):] = np.array(row)[:seq_length]
  return features

seq_length=200
features = pad_features(encoded_text, seq_length)
labels = df['label'].values

In [ ]:
features.shape

(44898, 200)

In [ ]:
# Dataloaders
# Split data (80% train, 10% validation, 10% test)
X_train, X_rem, y_train, y_rem = train_test_split(features, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=42)

In [ ]:
# tensordatasets
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

In [ ]:
batch_size = 64
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)

        self.supports_masking = True
        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0

        weight = torch.zeros(feature_dim, 1)
        nn.init.xavier_uniform_(weight)
        self.weight = nn.Parameter(weight)

        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))

    def forward(self, x, mask=None):
        feature_dim = self.feature_dim
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim),
            self.weight
        ).view(-1, step_dim)

        if self.bias:
            eij = eij + self.b

        eij = torch.tanh(eij)
        a = torch.exp(eij)

        if mask is not None:
            a = a * mask

        a = a / (torch.sum(a, 1, keepdim=True) + 1e-10)

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

In [ ]:
class FakeNewsClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers,
                            bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.attention = Attention(hidden_dim * 2, seq_length) # hidden_dim * 2 for bidirectional
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        lstm_output, (hidden, cell) = self.lstm(embedded)
        # lstm_output shape: (batch_size, seq_len, hidden_dim * 2)

        attention_output = self.attention(lstm_output)
        # attention_output shape: (batch_size, hidden_dim * 2)

        attention_output = self.dropout(attention_output)
        output = self.fc(attention_output)
        return output

In [47]:
# training loooop
# parameters
vocab_size = len(vocab_to_int)
embedding_dim=100
hidden_dim = 128
output_dim = 1 # Single output for binary classification
n_layers = 2
bidirectional = True
dropout = 0.5

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [48]:
model = FakeNewsClassifier(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout)
model.to(device)

FakeNewsClassifier(
  (embedding): Embedding(215891, 100, padding_idx=0)
  (lstm): LSTM(100, 128, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (attention): Attention()
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [49]:
# loss and criterion
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [50]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs).squeeze(1)
        loss = criterion(outputs, labels.float())

        loss.backward()
        optimizer.step()

In [52]:
model.eval()

FakeNewsClassifier(
  (embedding): Embedding(215891, 100, padding_idx=0)
  (lstm): LSTM(100, 128, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (attention): Attention()
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [53]:
# .validation
val_losses = []
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs).squeeze(1)
        val_loss = criterion(outputs, labels.float())
        val_losses.append(val_loss.item())

        predicted = torch.round(torch.sigmoid(outputs))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {loss.item():.4f}, Val Loss: {np.mean(val_losses):.4f}, Val Acc: {100*correct/total:.2f}%')

Epoch 5/5, Train Loss: 0.0056, Val Loss: 0.0095, Val Acc: 99.80%


In [54]:
def predict_news(model, text, vocab_to_int, seq_length):
    model.eval()
    with torch.no_grad():
        tokens = preprocess_text(text)
        encoded = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in tokens]
        padded_encoded = pad_features([encoded], seq_length)
        input_tensor = torch.from_numpy(padded_encoded).to(device)
        output = model(input_tensor).squeeze(1)
        prediction = torch.sigmoid(output).item() # Get the probability
        return prediction

# Example usage:
input_text = "This is a test news article about politics."
prediction = predict_news(model, input_text, vocab_to_int, seq_length)

print(f"The predicted probability of this news being fake is: {prediction:.4f}")
if prediction > 0.5:
    print("This news is likely fake.")
else:
    print("This news is likely true.")

The predicted probability of this news being fake is: 0.9992
This news is likely fake.
